        Total of 20 pages with a lot of sentences. (3 main Manchurian dishes)
        • Sentence tokenization
        • Every sentence to be marked as 0 or 1 based on if its an ingredient or not (nouns, numeric, measurement names , fractions)
        • If the sentence doesn’t belong to the ingredient section press Enter (which means you’ve entered a null string) (create a list to store the labels)
        • As entering will replace all null string with 0
        • Use for loop,
        • For every sentence there should be an input option which takes the label and add it to a list (0 or 1)
        • Later these null strings could be replaced by 0's
        • Then train with a classical NLP techniques that would distinguish if a sentence belongs to ingredient section or not

# Panneer
    https://www.vegrecipesofindia.com/paneer-manchurian-dry-recipe/
    https://www.indianhealthyrecipes.com/paneer-manchurian-recipe/
    https://hebbarskitchen.com/paneer-manchurian-dry-recipe/
    https://foodviva.com/chinese-recipes/paneer-manchurian-recipe/
    https://www.mydelicious-recipes.com/recipedetail/dry-paneer-manchurian
    https://www.tarladalal.com/paneer-in-manchurian-sauce-86r
    https://www.awesomecuisine.com/recipes/725/paneer-manchurian/
   
    https://www.yummytummyaarthi.com/paneer-manchurian-recipe-how-to-make/
    

# chicken
    https://www.indianhealthyrecipes.com/chicken-manchurian-recipe/
    https://recipes.timesofindia.com/recipes/chicken-manchurian/rs53241077.cms
    https://www.sanjeevkapoor.com/recipe/Chicken-Manchurian-Best-of-Chinese-Cooking.html
    https://food.ndtv.com/recipe-chicken-manchurian-chinese-1-465335
    https://www.archanaskitchen.com/healthy-chicken-manchurian-recipe
    https://ikneadtoeat.com/easy-indo-chinese-chicken-manchurian/
    https://www.kannammacooks.com/chicken-manchurian/

## Gobi 
    https://www.indianhealthyrecipes.com/gobi-manchurian-recipe/
    https://www.vegrecipesofindia.com/gobi-manchurian-dry-recipe/
    https://www.cookwithmanali.com/gobi-manchurian/
    https://hebbarskitchen.com/dry-gobi-manchurian-recipe/
    https://foodviva.com/chinese-recipes/gobi-manchurian-recipe/
    https://rainbowplantlife.com/gobi-manchurian/
    https://www.tarladalal.com/gobi-manchurian--how-to-make-gobi-manchurian-starter-40482r

In [31]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
stop_words = nltk.corpus.stopwords.words('english')
import pandas as pd

In [32]:
def text_extract_func(url):
    ####request the server fro the raw html content in text format and get just the text part of page
    r = requests.get(url).text
    text = ' '.join(BeautifulSoup(r, "html.parser").stripped_strings)
    
    ##sentence tokenize the entire para of text
    sent_tokenize= nltk.sent_tokenize(text)
    
    for i in range(len(sent_tokenize)):
        sent_tokenize[i] = sent_tokenize[i].lower()                   #converting all the words to lower case
        sent_tokenize[i] = re.sub(r'\W'," ",sent_tokenize[i])         #replacing . , others with space
        sent_tokenize[i] = re.sub(r'\s+'," ",sent_tokenize[i])        #replacing multiple space with single space
        sent_tokenize[i] = re.sub(r'\[[0-9]*\]',' ',sent_tokenize[i])
    return sent_tokenize

In [61]:
s = text_extract_func('https://www.yummytummyaarthi.com/paneer-manchurian-recipe-how-to-make/')

In [62]:
s

['paneer manchurian paneer manchurian recipe skip to primary navigation skip to main content skip to primary sidebar home recipe index about me masterchef journey travel advertisers media search menu icon home recipe index about me masterchef journey travel media advertisers facebook instagram pinterest twitter youtube subscribe search icon search home recipe index about me masterchef journey travel media advertisers facebook instagram pinterest twitter youtube home paneer manchurian recipe january 13 2022 by aarthi 9 comments twitter facebook 73 pinterest telegram whatsapp yummly 73 shares jump to recipe print recipe delicious paneer manchurian which taste so delicious with fried rice or noodles ',
 'pin table of contents 1 ',
 'what is a manchurian 2 ',
 'paneer manchurian paneer manchurian recipe recipe for paneer manchurian 3 ',
 'paneer manchurian gravy step by step 4 ',
 'healthy paneer manchurian dry paneer manchurian paneer manchurian my favourite ',
 'i love it to the core ',


In [35]:
def preprocess(sent):          
    sent = nltk.word_tokenize(sent)               ####work tokennize each sentence 
    sent = nltk.pos_tag(sent)                     ### tagging the parts of speech
    return sent

In [40]:
def ingredient_sent(sentence_list):                #function to figure out the sentence containing ingredients using the 
                                                   #cardinality concept
    l = []
    for i in sentence_list:                        ##fro each sentence call the preprocess function
        sent = preprocess(i)
        l.append(sent)                             ###append to a empty list l
    r = []
    c=0
    for i in range(len(l)):                         ### for each sentence in l
        for j in range(len(l[i])):                  ### for each word in one sentence of l
            if (l[i][j][-1] == 'CD'):               ##check if the pos tag is cardinality 
                #r.append(l[i][j])
                c =c + 1                            ###if yes increment counter
        if c > 10 :                                  ###check the condition for counter value 
            r.append(l[i])                          ### if satisfied append that sentence to the list r
        c = 0                                       ###reset counter
                                           

    f = [item for sublist in r for item in sublist]  ###flatening the nested lists
    np = []
    for n in range(len(f)):                          ###for each element in the flatened list f
        if f[n][0] not in stop_words:                ##check if the word is not a stopword
            if f[n][1] != 'CD':                      ###check if it is not  cardinality tagged 
        #if(f[n][1] == 'NN'):
                np.append(f[n][0])                  ###append to list np the string 
    ll = list(set(np))                              ###using set function and get the unique values
    n=[]
    for i in range(len(ll)):                        ##for each element in the unique list
            pattern = r'[0-9]'                      ##pattern search for numbers
            new_string = re.sub(pattern,'', ll[i])  ###if the string is a digit replace by empty string
            n.append(new_string)                    ###append to the list n
            n = ' '.join(n).split()                 ###removing the empty strings in the list n
            
    x=list()

    for data in s:                                  ##for each sentence in the list of sentences s
        c =  0
        for word in n:                              ###for each word in the list of words n
            if word in (data):                      ###check if the word is present in the sentence
                c = 1                               ###if yes set counter c = 1
                break                     
        x.append(c)                                 ###append to the list x
    
    sentence_label = [] 
    count= 0
    for data in s:                                  ##for each sentence in the list of sentences s
        sentence_label.append(  [data,x[count]]  )  ###appending the sentence with the counter value from x
        count = count + 1                           ##increment the index in the list x
    
    return sentence_label

In [64]:
r = ingredient_sent(s)  

In [42]:
import os
os.getcwd()

'C:\\Users\\Priyanka G\\CAPSTONE_PROJECT_WORK(2022)'

In [65]:
import pandas as pd

df = pd.DataFrame(r)
df['Sentence']=df[0]
df['Label'] = df[1]
df1 = df.drop([0,1], axis = 1)


In [66]:
df1

,Sentence,Label
0,paneer manchurian paneer manchurian recipe ski...,1
1,pin table of contents 1,1
2,what is a manchurian 2,0
3,paneer manchurian paneer manchurian recipe rec...,1
4,paneer manchurian gravy step by step 4,1
...,...,...
90,welcome,0
91,over the past few years i ve been on a mission...,1
92,i hope you enjoy the recipes on this blog as t...,1
93,more about me stay connected diwali sweets sna...,1


In [47]:
df = pd.DataFrame(r)
df['Sentence']=df[0]
df['Label'] = df[1]
df2 = df.drop([0,1], axis = 1)

In [51]:

df = pd.DataFrame(r)
df['Sentence']=df[0]
df['Label'] = df[1]
df3 = df.drop([0,1], axis = 1)
   

In [55]:

df = pd.DataFrame(r)
df['Sentence']=df[0]
df['Label'] = df[1]
df4 = df.drop([0,1], axis = 1)
   

In [59]:

df = pd.DataFrame(r)
df['Sentence']=df[0]
df['Label'] = df[1]
df5 = df.drop([0,1], axis = 1)
   

In [67]:
df2

,Sentence,Label
0,paneer manchurian 2 delicious recipes paneer m...,1
1,there are two ways to make this recipe paneer ...,1
2,you get the best of both these versions in thi...,1
3,it is up to you to make the dish as a dry snac...,1
4,what is paneer manchurian paneer manchurian re...,1
...,...,...
224,running the kitchen for decades i share tried ...,1
225,i also have a professional background in cooki...,1
226,learn more about dassana follow us facebook yo...,1
227,email it to a friend,1


In [68]:
df3

,Sentence,Label
0,paneer manchurian dry recipe how to make dry p...,1
1,a dry manchurian recipe prepared with paneer c...,1
2,an ideal party snack served as starter or as a...,0
3,paneer manchurian dry recipe how to make dry p...,1
4,basically an adapted version of chinese cuisin...,1
...,...,...
68,facebook twitter whatsapp pinterest email hebb...,0
69,easy to understand and make keep posting new r...,0
70,our other languages stay connected 9 052 334 f...,1
71,don t forget to follow us on social networks,0


In [69]:
df4

,Sentence,Label
0,paneer manchurian recipe dry spicy chinese man...,1
1,top 10 snacks breakfasts side dishes curries s...,1
2,this recipe is similar to gobi manchurian but ...,1
3,it is one of the easy paneer dishes of indo ch...,1
4,more chinese recipes momos chutney american ch...,1
...,...,...
136,the stepwise photos helped me a lot,0
137,jul 18 2012 by raj good recipe but can we add ...,1
138,primary sidebar like our recipes,1
139,follow us,0


In [71]:
# concatenating df1 and df2 along rows
vertical_concat = pd.concat([df1, df2,df3,df4,df5], axis=0)
vertical_concat

,Sentence,Label
0,paneer manchurian paneer manchurian recipe ski...,1
1,pin table of contents 1,1
2,what is a manchurian 2,0
3,paneer manchurian paneer manchurian recipe rec...,1
4,paneer manchurian gravy step by step 4,1
...,...,...
47,so you can either add or not,0
48,depending on your taste,0
49,reply brian march 16 2008 7 02 am i would like...,1
50,reply leave a comment cancel reply search reci...,1


In [70]:
df5

,Sentence,Label
0,paneer manchurian dry recipe paneer manchurian...,1
1,jump to recipe paneer manchurian is a perfect ...,1
2,cooking paneer with veggies is an indian tradi...,1
3,a combination of both these methods gives you ...,1
4,some people like to keep their manchurian dry ...,1
5,hot paneer manchurian can be relished with fri...,1
6,adding one egg white is optional as vegetarian...,0
7,however adding the egg white gives a smooth co...,1
8,add spring onions and celery to give it an aut...,1
9,you can also use tofu instead of paneer and th...,1


In [72]:
vertical_concat.to_csv('labelled_sentence.csv', index=False)